<a href="https://colab.research.google.com/github/alfawzaan/my-first-page/blob/master/Copy_of_Planta_Crop_Disease_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models


In [3]:
!git clone https://github.com/spMohanty/PlantVillage-Dataset

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 48.10 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (182401/182401), done.


In [0]:
# Transforms for the training data and testing data
train_transforms = transforms.Compose([
        transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Pass transforms in here, then run the next cell to see how the transforms look
data_dir = '/content/PlantVillage-Dataset/raw'
train_data = datasets.ImageFolder(data_dir + '/color', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/color', transform=test_transforms)


trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)



In [6]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = models.googlenet(pretrained=True)

# Build and train your network
# model = models.resnet152(pretrained=True)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 61.6MB/s]


In [19]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict

model.fc = nn.Sequential(OrderedDict([
                          ('dropout', nn.Dropout(0.5)),
                          ('fc1', nn.Linear(1024,512 )),
                          # ('relu', nn.ReLU()),
                          # ('fc2', nn.Linear(1024, 512)),
                          # ('relu', nn.ReLU()),
                          # ('fc3', nn.Linear(512, 256)),
                          # ('relu', nn.ReLU()),
                          # ('fc4', nn.Linear(256, 128)),
                          # ('relu', nn.ReLU()),
                          # ('fc5', nn.Linear(128, 64)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(512, 256)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(0.5)),
                          ('output', nn.Linear(256, 38))
                          #('output', nn.LogSoftmax(dim=1))
                          ]))
criterion = nn.CrossEntropyLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.0001)

model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [0]:
from datetime import datetime

In [0]:
epochs = 30
steps = 0
running_loss = 0
print_every = 5

now = datetime.now()
current_time = now.strftime("%d/%m/%Y %H:%M:%S")
print("Start Time =", current_time)
test_loss_min = np.Inf # track change in validation loss


for epoch in range(epochs):
  print(epoch)
  running_loss = 0

  for inputs, labels in trainloader:
    model.train()

    steps += 1

    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    optimizer.zero_grad()

    # start = time.time()

    logits = model(inputs)
    loss = criterion(logits, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  
    # print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

    # if steps % print_every == 0:
    test_loss = 0
    accuracy = 0
    model.eval()

  with torch.no_grad():
    for inputs, labels in testloader:
      inputs, labels = inputs.to(device), labels.to(device)
      logps = model(inputs)
      batch_loss = criterion(logps, labels)

      test_loss += batch_loss.item()

      # Calculate accuracy
      ps = torch.exp(logps)
      top_p, top_class = ps.topk(1, dim=1)
      equals = top_class == labels.view(*top_class.shape)
      accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

  print(f"Epoch {epoch+1}/{epochs}.. "
             f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader)*100:.3f}%")
   if test_loss <= test_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        test_loss_min,
        test_loss))
        torch.save(model.state_dict(), 'model_plant_diseases.pt')
        test_loss_min = test_loss
      
    # model.train()
now = datetime.now()
current_time = now.strftime("%d/%m/%Y %H:%M:%S")
print("End Time =", current_time)


Start Time = 16/01/2020 08:16:20
0
Epoch 1/30.. Train loss: 234.152.. Test loss: 0.499.. Test accuracy: 85.233%
1
Epoch 2/30.. Train loss: 180.210.. Test loss: 0.425.. Test accuracy: 87.971%
2
Epoch 3/30.. Train loss: 170.681.. Test loss: 0.426.. Test accuracy: 87.077%
3
